# The Plan

There are essentially two steps to this process:
1. The Generalized W state, of the form `|10..0> + |01..1> + .. + |00..1>` (unnormalized). This represents which element we are using. 
2. The Sampling operator, which samples a random point inside the chosen element, but since we are have a superposition of all elements, this will give us a superposition of all points inside elements (white pixels). 

In [28]:
desired_target, target = "quantinuum.sim.h1-1e", None

In [49]:
from azure.quantum.cirq import AzureQuantumService
from cirq.vis import plot_state_histogram
import cirq
from numpy import pi
from matplotlib import pyplot as plt

service = AzureQuantumService(
            resource_id = "/subscriptions/a5abf7fd-dcef-47af-8385-2fdf76385f21/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/qvc-2",
            location = "eastus")

for tgt in service.targets():
   if tgt.current_availability == "Available":
      if tgt.name == desired_target:
         print("-> " + tgt.name)
         target = tgt
      else:
         print(tgt.name)

ionq.qpu.aria-1
ionq.simulator
quantinuum.sim.h1-1sc
-> quantinuum.sim.h1-1e


### Step 1: The Generalized W State

In [75]:
# First things first, we need to be able to create the Generalized W state
# The most efficient way that I can think of to do this is as follows:
def generalized_w(lg_n: int, state: list[cirq.Qid], anc: list[cirq.Qid]):
    yield from (cirq.H(i) for i in anc)
    yield cirq.H(state[0])
    yield cirq.X(state[1])
    yield cirq.CX(state[0], state[1])
    size_before, index = 2, 0
    while size_before < (1 << lg_n):
        yield (
            cmove(anc[index], state[i], state[i + size_before])
            for i in range(size_before)
        )
        index += 1
        size_before *= 2

# This gate is equivalent to the Fredkin gate in the special
# case where the last qubit is known to be in the |0> state,
# up to a relative phase thanks to the use of a Margolus gate
def cmove(ctrl: cirq.Qid, q1: cirq.Qid, q2: cirq.Qid):
    yield from margolus(ctrl, q1, q2)
    yield cirq.CX(q2, q1)

# Equivalent to the Toffoli gate up to a relative phase
def margolus(ctrl1: cirq.Qid, ctrl2: cirq.Qid, q: cirq.Qid):
    yield cirq.Ry(rads = pi / 4)(q)
    yield cirq.CX(ctrl1, q)
    yield cirq.Ry(rads = pi / 4)(q)
    yield cirq.CX(ctrl2, q)
    yield cirq.Ry(rads = -pi / 4)(q)
    yield cirq.CX(ctrl1, q)
    yield cirq.Ry(rads = -pi / 4)(q)

In [76]:
# Let's test it out
lg_n = 2
state = cirq.LineQubit.range(1 << lg_n)
anc = cirq.LineQubit.range(len(state), len(state) + lg_n - 1)
wstate_circ = cirq.Circuit(generalized_w(lg_n, state, anc))
wstate_circ.append(cirq.measure(*state))
wstate_circ

0: ───H───────────@───────────────@─────────────────────────────────X─────────────────────────────────────────────────M───
                  │               │                                 │                                                 │
1: ───X───────────X───────────────┼─────────────────────────────────┼───────────@─────────────────────────────────X───M───
                                  │                                 │           │                                 │   │
2: ───Ry(0.25π)───X───Ry(0.25π)───X───Ry(-0.25π)───X───Ry(-0.25π)───@───────────┼─────────────────────────────────┼───M───
                  │                                │                            │                                 │   │
3: ───Ry(0.25π)───┼────────────────────────────────┼───X────────────Ry(0.25π)───X───Ry(-0.25π)───X───Ry(-0.25π)───@───M───
                  │                                │   │                                         │
4: ───H───────────@────────────────────────────────@───@─────────────────────────────────────────@────────────────────────

In [77]:
job = target.submit(wstate_circ, name="Generalized W State", repetitions=100)

In [93]:
job.status()

'Succeeded'

In [74]:
job.results()

{'m0': ['1101',
  '1100',
  '1111',
  '1101',
  '1100',
  '1101',
  '1110',
  '0000',
  '0001',
  '0000',
  '1110',
  '1111',
  '1110',
  '1100',
  '0000',
  '0011',
  '1100',
  '0010',
  '1101',
  '1100',
  '0010',
  '0001',
  '0010',
  '1100',
  '0000',
  '0010',
  '1110',
  '0110',
  '1100',
  '1110',
  '1110',
  '0011',
  '1110',
  '0000',
  '0011',
  '0011',
  '0011',
  '0001',
  '0011',
  '0010',
  '0011',
  '1101',
  '1100',
  '1101',
  '0000',
  '0011',
  '0001',
  '1101',
  '1101',
  '0010',
  '0010',
  '1101',
  '0000',
  '1111',
  '0010',
  '1111',
  '0010',
  '0001',
  '0000',
  '1101',
  '0011',
  '1110',
  '1100',
  '1101',
  '0000',
  '1101',
  '1100',
  '0010',
  '1101',
  '0011',
  '0010',
  '1110',
  '0011',
  '0000',
  '0011',
  '0010',
  '0000',
  '0000',
  '1100',
  '0011',
  '1110',
  '1110',
  '1110',
  '0000',
  '0001',
  '0011',
  '0010',
  '1101',
  '0011',
  '0010',
  '1100',
  '0001',
  '1100',
  '0000',
  '0011',
  '1111',
  '0000',
  '1101',
  '1110',
  '0